<a href="https://colab.research.google.com/github/shivammehta007/Information-Retrieval/blob/master/MLinRanking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Application of ML in Ranking

# Application of ML in Ranking of Search Engines

## Downloading Data From Kaggle
#### Might Take a while

In [30]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# Download Data
!pip install kaggle
import json
import os

with open('/content/drive/My Drive/kaggle.json') as kc:
    kaggle_config = json.load(kc)

os.environ['KAGGLE_USERNAME'] = kaggle_config['username'] 
os.environ['KAGGLE_KEY'] = kaggle_config['key'] 

# !kaggle competitions download -c text-relevance-competition-ir-2-itmo-fall-2019
# !unzip docs.tsv.zip
# !rm docs.tsv.zip

## Code

In [27]:
!pip install -U tqdm
!pip install rank_bm25
import os
import gzip
import nltk
import pickle
import pandas as pd
import logging
import itertools
import re
import time

import numpy as np
import pandas as pd

from rank_bm25 import BM25Okapi

from nltk.corpus import stopwords
nltk.download("stopwords")

from string import punctuation
from collections import defaultdict
from tqdm.notebook import tqdm

logging.basicConfig(level=logging.DEBUG)
logging.debug('Test Logger')

Requirement already up-to-date: tqdm in /usr/local/lib/python3.6/dist-packages (4.41.0)


DEBUG:root:Test Logger


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
indexing_data_location = '/content/docs.tsv'
queries = '/content/queries.numerate.txt'

### Building Inverted Index

#### Inverted Indexer Class

In [0]:
class InvertedIndexer:
    """
    This class makes inverted index
    """

    def __init__(self, filename=False):
        self.filename = filename
        self.stemmer_ru = nltk.SnowballStemmer("russian")
        self.stopwords = set(stopwords.words("russian")) | set(stopwords.words("english"))
        self.punctuation = punctuation # from string import punctuation
        if filename:
            self.inverted_index = self._build_index(self.filename)
        else:
            self.inverted_index = defaultdict(set)

    def preprocess(self, sentence):
        """
        Method to remove stop words and punctuations return tokens
        """
        NONTEXT = re.compile('[^0-9 a-z#+_а-яё]')

        sentence = sentence.lower()
        sentence = sentence.translate(str.maketrans('', '', punctuation))
        sentence = re.sub(NONTEXT,'',sentence)

        # Heavy Operation Taking lot of time will move it outside
        # tokens = [self.stemmer_ru.stem(word) for word in sentence.split()]

        tokens = [token for token in sentence.split() if token not in self.stopwords]

        return tokens

    def stem_keys(self, inverted_index):
        """
        Called after index is built to stem all the keys and normalize them
        """
        logging.debug('Indexing Complete will not Stem keys and remap indexes')
        temp_dict = defaultdict(set)
        i = 0
        for word in tqdm(inverted_index):

            stemmed_key = self.stemmer_ru.stem(word)
            temp_dict[stemmed_key].update(inverted_index[word])
            inverted_index[word] = None

        inverted_index = temp_dict
        logging.debug('Done Stemmping Indexes')
        return inverted_index

    def _build_index(self, indexing_data_location):
        """
        This method builds the inverted index and returns the invrted index dictionary
        """
        inverted_index = defaultdict(set)
        with open(indexing_data_location, "rb") as f:
            for line in tqdm(f):
                line = line.decode().split('\t')
                file_number = line[0]
                subject = line[1]
                text = line[2]
                line = subject + ' ' + text

                for word in self.preprocess(line):
                        inverted_index[word].add(int(file_number))

        # Intermediate Save
        # with open('/content/intermediate_inverted_index.pickle', mode='wb') as f:
        #     pickle.dump(inverted_index, f)
        # Collab crashed don't do that 

        inverted_index = self.stem_keys(inverted_index)

        return inverted_index

    def save(self, filename_to_save):
        """
        Save method to save the inverted indexes
        """
        with open(filename_to_save, mode='wb') as f:
            pickle.dump(self.inverted_index, f)

    def load(self, filelocation_to_load):
        """
        Load method to load the inverted indexes
        """
        with open(filelocation_to_load, mode='rb') as f:
            self.inverted_index = pickle.load(f)

#### SolutionPredictor Class

In [0]:
class SolutionPredictor:
    """
    This classes uses object of Hw1SolutionIndexer
    to make boolean search
    """
    def __init__(self, indexer):
        """
        indexer : object of class Hw1SolutionIndexer
        """
        self.indexer = indexer


    def find_docs(self, query):
        """
        This method provides booleaen search
        query : string with text of query
        Returns Python set with documents which contain query words
        Should return maximum 100 docs
        """
        tokens = self.indexer.preprocess(query)
        tokens = [self.indexer.stemmer_ru.stem(word) for word in tokens]
        docs_list = set()
        for word in tokens:
            if len(docs_list) > 0:
                docs_list.intersection_update(self.indexer.inverted_index[word])
            else:
                docs_list.update(self.indexer.inverted_index[word])

        return set(itertools.islice(docs_list, 100))

#### Generate Index 
Run Only when Index is not generated Otherwise use the Loading code block

In [7]:
logging.debug('Index is creating...')
start = time.time()
new_index = InvertedIndexer(indexing_data_location) 
end = time.time()
logging.debug('Index has been created and saved as inverted_index.pickle in {:.4f}s'.format(end-start))

DEBUG:root:Index is creating...


DEBUG:root:Indexing Complete will not Stem keys and remap indexes


DEBUG:root:Done Stemmping Indexes
DEBUG:root:Index has been created and saved as inverted_index.pickle in 1007.3685s


In [8]:
len(new_index.inverted_index)

2858862

In [0]:
new_index.save('inverted_index.pickle')

In [0]:
!cp inverted_index.pickle '/content/drive/My Drive/Homeworks/InformationRetrieval/inverted_index_4.pickle'

#### Load Index
Run when Index is not generated

In [0]:
index_location = '/content/drive/My Drive/Homeworks/InformationRetrieval/inverted_index_4.pickle'

In [9]:
logging.debug('Loading Index...')
start = time.time()
new_index = InvertedIndexer()
new_index.load(index_location)
end = time.time()
logging.debug('Index has been loaded from inverted_index.pickle in {:.4f}s'.format(end-start))

DEBUG:root:Loading Index...
DEBUG:root:Index has been loaded from inverted_index.pickle in 66.7793s


In [10]:
len(new_index.inverted_index)

2858862

##### Testing Block

In [0]:
test_sentence = 'бандиты боятся ли ментов'

In [0]:
boolean_model = SolutionPredictor(new_index)

In [13]:
start = time.time()
print(boolean_model.find_docs(test_sentence))
logging.debug('Search Time: {}'.format(time.time() - start))

DEBUG:root:Search Time: 0.0027985572814941406


{59393, 122885, 92166, 370697, 262166, 477206, 145432, 575512, 110616, 432154, 432166, 247847, 106540, 557108, 421941, 12351, 522305, 190534, 221260, 198735, 284753, 579665, 364632, 505945, 364635, 178274, 313443, 313451, 419951, 489583, 75893, 22646, 57466, 489595, 180351, 106626, 108677, 88199, 221320, 112780, 489618, 489620, 485525, 489622, 221335, 221336, 383136, 364705, 362658, 362662, 204967, 489639, 112810, 245933, 20656, 489651, 577718, 403642, 383163, 61629, 577727, 14529, 403650, 579780, 563400, 577738, 413900, 14540, 577743, 520399, 135378, 379091, 362708, 340183, 92376, 383196, 442590, 221414, 489710, 299250, 155891, 510199, 311545, 311553, 430350, 495888, 92435, 430358, 364825, 416028, 174377, 174380, 10544, 284977, 284978, 452915, 414004, 414006, 311610, 284990}


### Loading File to DataFrame
We are doing this for faster search based on index numbers

In [14]:
%%time
database = pd.read_csv(indexing_data_location, sep='\t', header=None,
                       names=['id', 'subject', 'content'],
                       dtype = {'id': int,'subject': str,
                            'content': str})

CPU times: user 47.6 s, sys: 10.4 s, total: 58 s
Wall time: 2min 8s


In [15]:
database.head()

,id,subject,content
0,24,БОЛЕЕТ ЗА ФУТБОЛ И ТРАХАЕТ ДОЧЬ СОФТ ОРГАНАЙЗЕ...,ГЛАВНАЯ НОВОСТИ КАРТА СОФТ ОРГАНАЙЗЕР СКАЧАТЬ ...
1,26,МОНГОЛЬСКАЯ ТХАНКА СОФТФОН,ГЛАВНАЯ НОВОСТИ КАРТА СОФТФОН GT МОНГОЛЬСКАЯ Т...
2,30,СМОТРЕТЬ ФИЛЬМЫ ОНЛАЙН БЕСПЛАТНО СМОТРЕТЬ КОНЦ...,ЗАГРУЗКА ПОЖАЛУЙСТА ПОДОЖДИТЕ ВНИМАНИЕ ОБНАРУЖ...
3,35,КАК ПРИГОТОВИТЬ ТОРТ РАФАЭЛЛО В ДОМАШНИХ УСЛОВ...,ՀԱՅԵՐԵՆ РУССКИЙ ПОЛИТИКА ВНУТРЕННИЙ ВНЕШНИЙ ЭК...
4,39,МИФЫ О ГВ BREASTFEEDING AM,TOGGLE NAVIGATION BREASTFEEDING AM ГОРЯЧАЯ ЛИН...


In [0]:
query = 'бандиты боятся ли ментов'
value_present = boolean_model.find_docs(query)
data = database[database['id'].isin(list(value_present))]
# data['content'] = data['subject'] + ' ' + data['content']
del data['subject']
data = data.set_index('id').T.to_dict()

In [0]:
data_new = data.set_index('id').T.to_dict()

In [42]:
i = 0
for key in data:
    print(key, data[key])
    i += 1
    if i == 5:
        break

10544 {'content': 'ЛЕОНИД БОРОДИН ЛОВУШКА ДЛЯ АДАМА ПОВЕСТЬ OCR И РЕДАКЦИЯ CHERNOV SERGEY CHERNOV OREL RU ОРЕЛ НОЯБРЬ 2006 Г ОСНОВАНО НА ИЗДАНИИ Л БОРОДИН ЛОВУШКА ДЛЯ АДАМА М РОМАН ГАЗЕТА 6 1996 Г ПОСВЯЩАЕТСЯ МАТЕРИ СОН С КОТОРОГО ВСЕ НАЧАЛОСЬ БЫЛ О МАМЕ ПРИ ЖИЗНИ Я НЕ ЗНАЛ ЕЕ ТАКОЙ НЕ ВИДЕЛ НЕ ПОМНИЛ НЕ ПОНИМАЛ А ЭТОТ ДОЛГИЙ СОН СОСТОЯЛ ИЗ ОДНОГО ПЕЧАЛЬНОГО ЕЕ ЛИЦА МЕНЯ САМОГО ТОЖЕ НЕ БЫЛО В СНОВИДЕНИИ МОЙ РАЗУМ ЛИШЬ ПРИСУТСТВОВАЛ КАК НЕЧТО БЕСТЕЛЕСНОЕ И ВОВСЕ БЕЗ ЛИЧНОСТИ БЕЗ ПРАВ И ЖЕЛАНИЙ НО С ЕДИНСТВЕННОЙ ФУНКЦИЕЙ ВОСПРИЯТИЯ ИТАК БЫЛО ОДНО ПЕЧАЛЬНОЕ ЛИЦО МОЕЙ МАМЫ И ОНО РАЗГОВАРИВАЛО СО МНОЙ СВОЕЙ ПЕЧАЛЬЮ СЛОВ НЕ БЫЛО ЭТО ПОТОМ ПРОСНУВШИСЬ Я ПЕРЕВЕЛ ВСЕ В СЛОВА И СМЫСЛЫ ЭТО ПОТОМ ВЕРНУВШИСЬ В СОБСТВЕННОЕ Я РАЗУМ МОЙ УЖАСНУЛСЯ ИЛИ ТОЧНЕЕ СООБЩИЛ УЖАС МОИМ ЧУВСТВАМ И ОНИ ЗАТРЕПЕТАЛИ ТО ЕСТЬ ЭТО Я ЗАТРЕПЕТАЛ И СЛЕЗЫ И ЗАРЫДАЛ БЫ ЕСЛИ БЫ ДАЛ СЕБЕ ВОЛЮ НО СЖАЛ ЗУБЫ И КУЛАКИ ТЕМ ПРЕДОТВРАТИВ ПОСТЫДНЫЕ КОНВУЛЬСИИ ГРУДИ ГОРЛА И ВСЕГО ПРОЧЕГО ЧТО ВОССОЗДАЕТ И СОТВОРЯЕТ ЖАЛКОЕ СОСТОЯНИЕ ПЛА